# Introduction

This Notebooks is a join notebook from both the prepare_data and pytorch-bst in order to be run in google colab.

# Prepare data section

In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch
import pytorch_lightning as pl
from tqdm import tqdm
import torchmetrics
import math
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import torch.nn as nn
import numpy as np
from math import sqrt

In [ ]:
!nvidia-smi

Wed May 25 05:25:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.__version__

'1.11.0+cu113'

## Settings

In [ ]:
WINDOW_SIZE = 20

## Data

In [ ]:
raw_df = pd.read_csv('Office_Products.csv', usecols=['rating', 'reviewerID', 'product_id', 'date'])

In [ ]:
# Prototyping 목적으로 50만 개의 행만 이용
raw_df = raw_df.iloc[:500000, :]
raw_df.rename(columns={'reviewerID': 'user_id'}, inplace=True)
raw_df.loc[ :, 'rating'] = raw_df.loc[:, 'rating'].apply(lambda x: float(x))
raw_df.head()

,rating,user_id,product_id,date
0,3.0,A2WJLOXXIB7NF3,0140503528,1162512000
1,5.0,A1RKICUK0GG6VF,0140503528,1147132800
2,5.0,A1QA5E50M398VW,0140503528,1142035200
3,5.0,A3N0HBW8IP8CZQ,0140503528,980294400
4,5.0,A1K1JW1C5CUSUZ,0140503528,964915200


In [ ]:
item_sizes = raw_df.groupby('product_id').size()
good_items = item_sizes.index[item_sizes >= 2]
raw_df = raw_df[raw_df['product_id'].isin(good_items)]


user_sizes = raw_df.groupby('user_id').size()
good_users = user_sizes.index[user_sizes >= 5]
raw_df = raw_df[raw_df['user_id'].isin(good_users)]
print(len(raw_df))

21721


In [ ]:
umap = {u: i for i, u in enumerate(set(raw_df['user_id']))}
smap = {s: i for i, s in enumerate(set(raw_df['product_id']))}
raw_df['user_id'] = raw_df['user_id'].map(umap)
raw_df['product_id'] = raw_df['product_id'].map(smap)
raw_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,rating,user_id,product_id,date
302,3.0,1139,1178,1476057600
345,4.0,980,1178,1448755200
458,3.0,1139,785,1476057600
494,4.0,980,785,1448755200
577,5.0,719,785,1368489600


## Make Sequence

In [ ]:
df_group = raw_df.sort_values(by=['date']).groupby('user_id')

df = pd.DataFrame(
    data={
        'user_id': list(df_group.groups.keys()),
        'product_id': list(df_group.product_id.apply(list)),
        'rating': list(df_group.rating.apply(list)),
        'date': list(df_group.date.apply(list)),
    }
)

In [ ]:
df.head()

,user_id,product_id,rating
0,0,"[[557, 1471, 1142, 125, 837], [557, 1471, 1142...","[[5.0, 4.0, 4.0, 5.0, 4.0], [5.0, 4.0, 4.0, 5...."
1,1,"[[752, 775, 417, 1519, 1519], [775, 417, 1519,...","[[5.0, 5.0, 5.0, 5.0, 5.0], [5.0, 5.0, 5.0, 5...."
2,2,"[[719, 1775, 1946, 822, 2295], [1775, 1946, 82...","[[5.0, 5.0, 5.0, 5.0, 5.0], [5.0, 5.0, 5.0, 5...."
3,3,"[[2017, 1766, 2011, 497, 486], [2017, 1766, 20...","[[5.0, 5.0, 5.0, 5.0, 3.0], [5.0, 5.0, 5.0, 5...."
4,4,"[[2265, 1522, 1940, 1071, 1131], [1522, 1940, ...","[[5.0, 5.0, 5.0, 5.0, 5.0], [5.0, 5.0, 5.0, 5...."


In [ ]:
sequence_length = 5
step_size = 1


def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        if len(seq) < window_size:
            seq = values[-window_size:]
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


df.product_id = df.product_id.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

df.rating = df.rating.apply(
    lambda ids: create_sequences(ids, sequence_length, step_size)
)

del df['date']

In [ ]:
df_transformed = df.explode(['product_id', 'rating'], ignore_index=True)
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12169 entries, 0 to 12168
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     12169 non-null  int64 
 1   product_id  12169 non-null  object
 2   rating      12169 non-null  object
dtypes: int64(1), object(2)
memory usage: 285.3+ KB


In [ ]:
df_transformed.dropna(axis=0, how='any', inplace=True)
df_transformed.isnull().any()

user_id       False
product_id    False
rating        False
dtype: bool

In [ ]:
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12169 entries, 0 to 12168
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     12169 non-null  int64 
 1   product_id  12169 non-null  object
 2   rating      12169 non-null  object
dtypes: int64(1), object(2)
memory usage: 380.3+ KB


In [ ]:
df_transformed

,user_id,product_id,rating
0,0,"[557, 1471, 1142, 125, 837]","[5.0, 4.0, 4.0, 5.0, 4.0]"
1,0,"[557, 1471, 1142, 125, 837]","[5.0, 4.0, 4.0, 5.0, 4.0]"
2,1,"[752, 775, 417, 1519, 1519]","[5.0, 5.0, 5.0, 5.0, 5.0]"
3,1,"[775, 417, 1519, 1519, 775]","[5.0, 5.0, 5.0, 5.0, 5.0]"
4,1,"[775, 417, 1519, 1519, 775]","[5.0, 5.0, 5.0, 5.0, 5.0]"
...,...,...,...
12164,3183,"[1637, 1170, 865, 226, 1054]","[5.0, 4.0, 5.0, 4.0, 4.0]"
12165,3183,"[1170, 865, 226, 1054, 908]","[4.0, 5.0, 4.0, 4.0, 4.0]"
12166,3183,"[865, 226, 1054, 908, 1341]","[5.0, 4.0, 4.0, 4.0, 2.0]"
12167,3183,"[226, 1054, 908, 1341, 286]","[4.0, 4.0, 4.0, 2.0, 5.0]"


In [ ]:
df_transformed.product_id = df_transformed.product_id.apply(lambda x: ",".join([str(v) for v in x]))
df_transformed.rating = df_transformed.rating.apply(lambda x: ",".join([str(v) for v in x]))
df_transformed.rename(columns={'product_id': 'seq_product_ids', 'rating': 'seq_ratings'}, inplace=True)

In [ ]:
random_selection = np.random.rand(len(df_transformed.index)) <= 0.85
train_data = df_transformed[random_selection]
test_data = df_transformed[~random_selection]

train_data.to_csv('train_data.csv', index=False, sep='|')
test_data.to_csv('test_data.csv', index=False, sep='|')

In [ ]:
train_data

,user_id,seq_product_ids,seq_ratings
0,0,"557,1471,1142,125,837","5.0,4.0,4.0,5.0,4.0"
1,0,"557,1471,1142,125,837","5.0,4.0,4.0,5.0,4.0"
3,1,"775,417,1519,1519,775","5.0,5.0,5.0,5.0,5.0"
4,1,"775,417,1519,1519,775","5.0,5.0,5.0,5.0,5.0"
6,2,"1775,1946,822,2295,564","5.0,5.0,5.0,5.0,5.0"
...,...,...,...
12160,3181,"654,822,1699,2221,1024","5.0,2.0,5.0,5.0,5.0"
12162,3182,"1065,1343,286,908,1054","5.0,5.0,5.0,5.0,5.0"
12164,3183,"1637,1170,865,226,1054","5.0,4.0,5.0,4.0,4.0"
12166,3183,"865,226,1054,908,1341","5.0,4.0,4.0,4.0,2.0"


In [ ]:
train_data.iloc[0].user_id

0

# BST Implementation and training

In [ ]:
import pandas as pd
import torch
import pytorch_lightning as pl
from tqdm import tqdm
import torchmetrics
import math
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import torch.nn as nn
import numpy as np
from torchtext.vocab import vocab
from collections import Counter

## Pytorch dataset

In [ ]:
data_keys = {
    'user_id': list(df_transformed.user_id.unique()),
    'product_id': list(raw_df.product_id.unique()), #이렇게 해야 개수가 훨 줄어듬
}

In [ ]:
len(data_keys['user_id'])

3184

In [ ]:
import pandas as pd
import torch
import torch.utils.data as data
from torchvision import transforms
import ast
from torch.nn.utils.rnn import pad_sequence

class AmazonDataset(data.Dataset):
    """Movie dataset."""

    def __init__(
        self, ratings_file,test=False
    ):
        """
        Args:
            csv_file (string): Path to the csv file with user,past,future.
        """
        self.ratings_frame = pd.read_csv(
            ratings_file,
            delimiter="|",
            # iterator=True,
        )
        self.test = test

    def __len__(self):
        return len(self.ratings_frame)

    def __getitem__(self, idx):
        data = self.ratings_frame.iloc[idx]
        user_id = int(data.user_id)
        product_history = list(map(int, data.seq_product_ids.split(',')))
        product_history_ratings = list(map(float, data.seq_ratings.split(',')))
        target_product_id = product_history[-1:][0]
        target_product_rating = product_history_ratings[-1:][0]
        
        product_history = torch.LongTensor(product_history[:-1])
        product_history_ratings = torch.FloatTensor(product_history_ratings[:-1])

        #target_id는 encoding no

        
        # sex = data.sex
        # age_group = data.age_group
        # occupation = data.occupation
        
        return (user_id, product_history, target_product_id,  product_history_ratings), target_product_rating#, sex, age_group, occupation

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
"""
  BST Model
"""

class PositionalEmbedding(nn.Module):
    """
    Computes positional embedding following "Attention is all you need"
    """

    def __init__(self, max_len, d_model):
        super().__init__()

        # Compute the positional encodings once in log space.
        self.pe = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size = x.size(0)
        return self.pe.weight.unsqueeze(0).repeat(batch_size, 1, 1)


class BST(pl.LightningModule):
    def __init__(
        self, args=None
    ):
        super().__init__()
        super(BST, self).__init__()
        
        self.save_hyperparameters()
        self.args = args
        #-------------------
        # Embedding layers
        ##Users 
        self.embeddings_user_id = nn.Embedding(
            len(data_keys['user_id'])+2, int(math.sqrt(len(data_keys['user_id'])))+1
        )


        self.embeddings_position  = nn.Embedding(
           sequence_length, int(math.sqrt(len(data_keys['product_id'])))
        )

        self.embeddings_product_id = nn.Embedding(
            len(data_keys['product_id'])+2, int(math.sqrt(len(data_keys['product_id'])))
        )
        
        
        self.positional_embedding = PositionalEmbedding(sequence_length, 63)
        
        # Network
        self.transfomerlayer = nn.TransformerEncoderLayer(63, 3, dropout=0.2)
        self.linear = nn.Sequential(
            nn.Linear(
                372,
                256,
            ),
            # nn.LeakyReLU(),
            # nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 1),
        )
        self.criterion = torch.nn.MSELoss()
        self.mae = torchmetrics.MeanAbsoluteError()
        self.mse = torchmetrics.MeanSquaredError()
        


    def encode_input(self,inputs):
        (user_id, product_history, target_product_id,  product_history_ratings), target_product_rating = inputs
               
        #MOVIES
        product_history = self.embeddings_product_id(product_history)
        target_product = self.embeddings_product_id(target_product_id)
        
        positions = torch.arange(0,sequence_length-1,1,dtype=int,device=self.device)
        positions = self.embeddings_position(positions)

        
        encoded_sequence_products_with_poistion_and_rating = (product_history + positions) *  product_history_ratings.reshape(-1, sequence_length-1,1)
        
        target_product = torch.unsqueeze(target_product, 1)
        transfomer_features = torch.cat((encoded_sequence_products_with_poistion_and_rating, target_product),dim=1)
        
        #USERS
        user_features = self.embeddings_user_id(user_id)
        
        return transfomer_features, user_features, target_product_rating.float()
    
    def forward(self, batch):
        transfomer_features, user_features, target_product_rating = self.encode_input(batch)
        transfomer_features = self.positional_embedding(transfomer_features)
        transformer_output = self.transfomerlayer(transfomer_features)
        transformer_output = torch.flatten(transformer_output,start_dim=1)
        
        #Concat with other features
        features = torch.cat((transformer_output,user_features),dim=1)
        output = self.linear(features)
        return output, target_product_rating
        
    def training_step(self, batch, batch_idx):
        out, target_product_rating = self(batch)
        out = out.flatten()
        loss = self.criterion(out, target_product_rating)
        
        mae = self.mae(out, target_product_rating)
        mse = self.mse(out, target_product_rating)
        rmse =torch.sqrt(mse)
        self.log(
            "train/mae", mae, on_step=True, on_epoch=False, prog_bar=False
        )
        
        self.log(
            "train/rmse", rmse, on_step=True, on_epoch=False, prog_bar=False
        )
        
        self.log("train/step_loss", loss, on_step=True, on_epoch=False, prog_bar=False)
        return loss
    
    def validation_step(self, batch, batch_idx):
        out, target_product_rating = self(batch)
        out = out.flatten()
        loss = self.criterion(out, target_product_rating)
        
        mae = self.mae(out, target_product_rating)
        mse = self.mse(out, target_product_rating)
        rmse =torch.sqrt(mse)
        
        return {"val_loss": loss, "mae": mae.detach(), "rmse":rmse.detach()}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_mae = torch.stack([x["mae"] for x in outputs]).mean()
        avg_rmse = torch.stack([x["rmse"] for x in outputs]).mean()
        
        print("epoch end: 3losses are {0}, {1}, {2}".format(avg_loss, avg_mae, avg_rmse))
        self.log("val/loss", avg_loss, on_step=False, on_epoch=True, prog_bar=False)
        self.log("val/mae", avg_mae, on_step=False, on_epoch=True, prog_bar=False)
        self.log("val/rmse", avg_rmse, on_step=False, on_epoch=True, prog_bar=False)

    def test_epoch_end(self, outputs):
        users = torch.cat([x["users"] for x in outputs])
        y_hat = torch.cat([x["top14"] for x in outputs])
        users = users.tolist()
        y_hat = y_hat.tolist()
        
        data = {"users": users, "top14": y_hat}
        df = pd.DataFrame.from_dict(data)
        print(len(df))
        df.to_csv("lightning_logs/predict.csv", index=False)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=0.0005)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument("--learning_rate", type=float, default=0.01)
        return parser

    ####################
    # DATA RELATED HOOKS
    ####################

    def setup(self, stage=None):
        print("Loading datasets")
        self.train_dataset = AmazonDataset("./train_data.csv")
        self.val_dataset = AmazonDataset("./test_data.csv")
        self.test_dataset = AmazonDataset("./test_data.csv")
        print("Done")

    def train_dataloader(self):
        print("fine?")
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=128,
            shuffle=True,
            num_workers=os.cpu_count(),
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=128,
            shuffle=False,
            num_workers=os.cpu_count(),
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=128,
            shuffle=False,
            num_workers=os.cpu_count(),
        )
        
model = BST()
trainer = pl.Trainer(gpus=1,max_epochs=50)
trainer.fit(model)

In [ ]:
from easydict import EasyDict
args = EasyDict({
    "seq_len": 4,
    "num_items": len(data_keys['product_id']),
    "num_users": len(data_keys['user_id']),
    "bert_num_blocks": 2,
    "bert_num_heads": 4,
    "bert_hidden_units": 128,
    "bert_dropout": 0.1,
    "model_init_seed": 0
})

In [ ]:
args

{'bert_dropout': 0.1,
 'bert_hidden_units': 128,
 'bert_num_blocks': 2,
 'bert_num_heads': 4,
 'model_init_seed': 0,
 'num_items': 2354,
 'num_users': 3184,
 'seq_len': 4}

In [ ]:
from models.bert import BERTModel

"""
  Bert4rec
"""


class Bert4rec(pl.LightningModule):
    def __init__(
        self, args=None
    ):
        super().__init__()
        super(Bert4rec, self).__init__()
        
        self.save_hyperparameters()
        self.args = args
        #-------------------
        # Embedding layers
        self.bert_model = BERTModel(args)
        self.criterion = torch.nn.MSELoss()
        self.mae = torchmetrics.MeanAbsoluteError()
        self.mse = torchmetrics.MeanSquaredError()
        

    
    def forward(self, batch):
        inputs, target_product_rating = batch
        output = self.bert_model(inputs)
        return output, target_product_rating.float()
        
    def training_step(self, batch, batch_idx):
        out, target_product_rating = self(batch)
        out = out.flatten()
        loss = self.criterion(out, target_product_rating)
        
        mae = self.mae(out, target_product_rating)
        mse = self.mse(out, target_product_rating)
        rmse =torch.sqrt(mse)
        self.log(
            "train/mae", mae, on_step=True, on_epoch=False, prog_bar=False
        )
        
        self.log(
            "train/rmse", rmse, on_step=True, on_epoch=False, prog_bar=False
        )
        
        self.log("train/step_loss", loss, on_step=True, on_epoch=False, prog_bar=False)
        return loss
    
    def validation_step(self, batch, batch_idx):
        out, target_product_rating = self(batch)
        out = out.flatten()
        loss = self.criterion(out, target_product_rating)
        
        mae = self.mae(out, target_product_rating)
        mse = self.mse(out, target_product_rating)
        rmse =torch.sqrt(mse)
        
        return {"val_loss": loss, "mae": mae.detach(), "rmse":rmse.detach()}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_mae = torch.stack([x["mae"] for x in outputs]).mean()
        avg_rmse = torch.stack([x["rmse"] for x in outputs]).mean()
        
        print("epoch end: 3losses are {0}, {1}, {2}".format(avg_loss, avg_mae, avg_rmse))
        self.log("val/loss", avg_loss, on_step=False, on_epoch=True, prog_bar=False)
        self.log("val/mae", avg_mae, on_step=False, on_epoch=True, prog_bar=False)
        self.log("val/rmse", avg_rmse, on_step=False, on_epoch=True, prog_bar=False)

    def test_epoch_end(self, outputs):
        users = torch.cat([x["users"] for x in outputs])
        y_hat = torch.cat([x["top14"] for x in outputs])
        users = users.tolist()
        y_hat = y_hat.tolist()
        
        data = {"users": users, "top14": y_hat}
        df = pd.DataFrame.from_dict(data)
        print(len(df))
        df.to_csv("lightning_logs/predict.csv", index=False)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=0.0005)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument("--learning_rate", type=float, default=0.01)
        return parser

    ####################
    # DATA RELATED HOOKS
    ####################

    def setup(self, stage=None):
        print("Loading datasets")
        self.train_dataset = AmazonDataset("./train_data.csv")
        self.val_dataset = AmazonDataset("./test_data.csv")
        self.test_dataset = AmazonDataset("./test_data.csv")
        print("Done")

    def train_dataloader(self):
        print("fine?")
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=128,
            shuffle=True,
            num_workers=os.cpu_count(),
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=128,
            shuffle=False,
            num_workers=os.cpu_count(),
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=128,
            shuffle=False,
            num_workers=os.cpu_count(),
        )
        
model = Bert4rec(args=args)
trainer = pl.Trainer(gpus=1,max_epochs=50)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Loading datasets
Done


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | bert_model | BERTModel         | 1.1 M 
1 | criterion  | MSELoss           | 0     
2 | mae        | MeanAbsoluteError | 0     
3 | mse        | MeanSquaredError  | 0     
-------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.574     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

epoch end: 3losses are 22.308856964111328, 4.663369655609131, 4.722982406616211
fine?


...

epoch end: 3losses are 0.28528183698654175, 0.2565035820007324, 0.5183299779891968


Validation: 0it [00:00, ?it/s]

epoch end: 3losses are 0.28935709595680237, 0.3072558045387268, 0.5223156809806824


Validation: 0it [00:00, ?it/s]

epoch end: 3losses are 0.2949879765510559, 0.3417607843875885, 0.5311439037322998


In [ ]:
Sdef 

SyntaxError: ignored